## XGB model
This notebooks trains the XGBoost model required for the experiment

In [1]:
import os
import pandas as pd
import glob

In [2]:
# first, prep data

In [3]:
scratch_dir = '/hot/scratch/xgb_flights/'
flights_root = '/hot/data/flights_all/'

In [4]:
paths = glob.glob(os.path.join(flights_root, 'flights_on_time*.csv'))

In [5]:
target_vars = ['ARR_DELAY', 'DEP_DELAY', 'CARRIER_DELAY',
               'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
               'LATE_AIRCRAFT_DELAY']

In [6]:
decision_vars = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK',
                 'OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID',
                 'DISTANCE', 'CRS_ARR_TIME', 'CRS_DEP_TIME', 'OP_CARRIER_FL_NUM']

In [7]:
# use years with data! 2003 and june!

In [8]:
path = paths[0]
def has_data(path):
    year, month = int(path[path.find('mance_')+6:-7]), int(path[path.find('mance_')+11:-4])
    if year > 2003:
        return True
    return year == 2003 and month >= 6

In [9]:
paths_with_data = sorted(list(filter(has_data, paths)))

In [10]:
path = paths_with_data[0]

In [11]:
df = pd.read_csv(path)

In [12]:
len(df)

536496

In [13]:
valid_idx = df[target_vars].dropna().index

In [14]:
df = df.loc[valid_idx]
len(df)

89441

In [15]:
# remove diverted and cancelled flights. => they get special treatment.
len(df[(df['DIVERTED'] != 1.0) & (df['CANCELLED'] != 1.0)])

89441

In [16]:
# filter out to what is only needed
dest_preprocess_dir = os.path.join(scratch_dir, 'prefiltered')
os.makedirs(dest_preprocess_dir, exist_ok=True)

In [17]:
from tqdm import tqdm

In [104]:
# preprocess csv files and write out!
for path in tqdm(paths_with_data[:2]):
    df = pd.read_csv(path, usecols=target_vars + decision_vars)
    
    valid_idx = df[target_vars].dropna().index
    df = df.loc[valid_idx]
    df.to_csv(os.path.join(dest_preprocess_dir, os.path.basename(path)), index=None)

100%|██████████| 2/2 [00:09<00:00,  4.86s/it]


In [18]:
# load a single file and make an XGB boost out of it!
paths = sorted(glob.glob(os.path.join(dest_preprocess_dir, 'flights_on_*.csv')))

path = paths[0]

In [75]:
df = pd.read_csv(path)

In [76]:
categorical_vars = ['OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'OP_CARRIER_FL_NUM']

In [82]:
df[df['ARR_DELAY'] > 0.0]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2,6,5,4,XE,2279,11049,12266,1655,42.0,1736,55.0,74.0,0.0,0.0,13.0,0.0,42.0
1,2,6,12,4,XE,2379,11049,12266,1655,26.0,1736,18.0,74.0,0.0,0.0,0.0,0.0,18.0
2,2,6,16,1,XE,2379,11049,12266,1655,118.0,1736,112.0,74.0,0.0,0.0,0.0,0.0,112.0
3,2,6,18,3,XE,2379,11049,12266,1655,-9.0,1736,17.0,74.0,0.0,0.0,17.0,0.0,0.0
4,2,6,23,1,XE,2379,11049,12266,1655,44.0,1736,39.0,74.0,0.0,39.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89436,2,6,8,7,OO,6965,12892,14893,1300,20.0,1425,18.0,373.0,18.0,0.0,0.0,0.0,0.0
89437,2,6,9,1,OO,3601,14869,14113,1855,13.0,1947,30.0,150.0,30.0,0.0,0.0,0.0,0.0
89438,2,6,9,1,OO,3601,14113,14869,2015,19.0,2103,22.0,150.0,22.0,0.0,0.0,0.0,0.0
89439,2,6,9,1,OO,3613,14869,15389,1845,9.0,1941,21.0,175.0,21.0,0.0,0.0,0.0,0.0


In [21]:
df_encoded = pd.get_dummies(df, columns=categorical_vars, sparse=True)

In [22]:
df_encoded.columns

Index(['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'CRS_DEP_TIME',
       'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'DISTANCE', 'CARRIER_DELAY',
       ...
       'OP_CARRIER_FL_NUM_7984', 'OP_CARRIER_FL_NUM_7985',
       'OP_CARRIER_FL_NUM_7986', 'OP_CARRIER_FL_NUM_7987',
       'OP_CARRIER_FL_NUM_7988', 'OP_CARRIER_FL_NUM_7989',
       'OP_CARRIER_FL_NUM_7993', 'OP_CARRIER_FL_NUM_7996',
       'OP_CARRIER_FL_NUM_7997', 'OP_CARRIER_FL_NUM_7999'],
      dtype='object', length=6221)

In [23]:
y = df_encoded[sorted(target_vars)].values

In [24]:
X = df_encoded[sorted(list(set(df_encoded.columns) - set(target_vars)))].values

In [25]:
X.shape

(89441, 6214)

In [59]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [88]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor, Ridge

In [61]:
#scaler = preprocessing.StandardScaler().fit(X_train)
scaler = preprocessing.MinMaxScaler().fit(X_train)

In [62]:
X_scaled = scaler.transform(X_train)

In [63]:
X_scaled

array([[0.6836302 , 0.6393373 , 0.93103448, ..., 0.        , 0.        ,
        0.        ],
       [0.80873622, 0.72854715, 0.34482759, ..., 0.        , 0.        ,
        0.        ],
       [0.60687023, 0.52676296, 0.37931034, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.93681086, 0.89804588, 0.62068966, ..., 0.        , 0.        ,
        0.        ],
       [0.38804071, 0.30586236, 0.10344828, ..., 0.        , 0.        ,
        0.        ],
       [0.89312977, 0.80926083, 0.96551724, ..., 0.        , 0.        ,
        0.        ]])

In [64]:
# reg = LinearRegression().fit(X_scaled, y=Y_train)

In [107]:
reg = Ridge(alpha=1.7).fit(X_scaled, y=Y_train)

In [108]:
reg.score(X_scaled, Y_train)

0.156991367549764

In [109]:
X_test_scaled = scaler.transform(X_test)

In [110]:
reg.score(X_test_scaled, Y_test)

-0.010687477674678068

In [111]:
Y_predicted = reg.predict(X_test_scaled)

In [112]:
Y_predicted

array([[ 6.07181804e+01,  3.15988755e+01,  5.88679918e+01, ...,
         1.07871885e+01,  3.02501340e-01,  4.34243728e+00],
       [ 3.89157000e+01,  1.42485682e+01,  3.10930842e+01, ...,
         1.01853118e+01,  1.17588148e-01,  1.42038222e+00],
       [ 6.52723931e+01,  8.12365172e-01,  3.86327255e+01, ...,
         3.03129215e+01, -1.12833680e-01,  2.20552917e+01],
       ...,
       [ 3.56716257e+01,  1.24365268e+01,  2.96180133e+01, ...,
         8.55545848e+00,  3.56267913e+00,  4.16232816e-01],
       [ 6.02485080e+01,  3.80774444e+00,  5.96593728e+01, ...,
         9.02129556e+00,  3.60564431e+00,  1.75512534e+01],
       [ 5.54802245e+01,  1.62977599e+01,  5.44975685e+01, ...,
         1.67497742e+01,  7.76261958e-02,  5.49098253e-02]])

In [113]:
reg.coef_

array([[ 6.12102217, 15.26873904, -1.88204671, ..., -4.49704184,
        -2.17346662,  0.        ],
       [-8.88124159, -7.64755863,  1.73145289, ..., -7.19481358,
         8.21272971,  0.        ],
       [ 0.87327443, 29.65477749,  0.24368729, ..., -8.18742599,
        -2.10669382,  0.        ],
       ...,
       [ 7.85595849, -8.93940076, -3.00390619, ..., 11.42187269,
        -3.02683106,  0.        ],
       [-0.03798717,  0.05805815,  0.49969238, ...,  0.27979919,
         2.28530955,  0.        ],
       [ 0.8980447 ,  3.47820828, -2.26311448, ..., -3.42283431,
        -1.56959438,  0.        ]])

In [114]:
!pip3 install mc2gen

Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/usr/lib/python3/dist-packages/pip/commands/install.py", line 353, in run
    wb.build(autobuilding=True)
  File "/usr/lib/python3/dist-packages/pip/wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 554, in _prepare_file
    require_hashes
  File "/usr/lib/python3/dist-packages/pip/req/req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 465, in find_requirement
    all_candidates = self.find_all_candidates(req.name)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 423, in fin

In [116]:
# This requires python >= 3.7.

In [72]:
from sklearn.metrics import mean_absolute_error

In [74]:
mean_absolute_error(Y_test, Y_predicted, multioutput='raw_values')

array([4.15907741e+10, 3.04502128e+10, 2.71152544e+10, 1.29126493e+10,
       1.55619486e+10, 1.09396865e+10, 5.39630281e+09])

In [55]:
Y_train

array([[28.,  0., -4., ..., 28.,  0.,  0.],
       [53.,  3., 54., ...,  0.,  0.,  0.],
       [43.,  0., 16., ..., 27.,  0.,  0.],
       ...,
       [53.,  0., 35., ..., 18.,  0.,  4.],
       [65.,  0., 40., ..., 25.,  0., 31.],
       [42.,  0., 49., ...,  0.,  0.,  0.]])

In [27]:
X_train.shape

(71552, 6214)

In [28]:
Y_train.shape

(71552, 7)

In [29]:
# following only works for newer XGBoost (> 1.6.0)

In [46]:
reg = xgb.XGBRegressor(verbosity=2,
                       eval_metric='mean_absolute_error',
                       num_target=Y_train.shape[1])

In [31]:
# older xgboost: train separate regressors...
#https://stackoverflow.com/questions/39540123/muti-output-regression-in-xgboost

In [32]:
from sklearn.multioutput import MultiOutputRegressor

In [35]:
multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', 
                                                             verbosity=2,
                                                            )).fit(X_train, Y_train)


KeyboardInterrupt: 

In [ ]:
# squared error
print(np.mean((multioutputregressor.predict(X_test) - Y_test)**2, axis=0))

In [ ]:
!pip3 install xgboost>1.6.0

In [48]:
reg.fit(X_train, Y_train, eval_set=[(X_test, Y_test)])

ValueError: Invalid shape: (71552, 7) for label

In [40]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

ValueError: Invalid shape: (71552, 7) for label

In [ ]:
param = {
    'eta': 0.3, 
    'booster': 'gblinear',
    'verbosity':2
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': len(target_vars),
    'eval_metric': 'rmse'} 

steps = 20  # The number of training iterations

model = xgb.train(param, D_train, steps)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

In [109]:
help(pd.get_dummies)

Help on function get_dummies in module pandas.core.reshape.reshape:

get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None) -> 'DataFrame'
    Convert categorical variable into dummy/indicator variables.
    
    Parameters
    ----------
    data : array-like, Series, or DataFrame
        Data of which to get dummy indicators.
    prefix : str, list of str, or dict of str, default None
        String to append DataFrame column names.
        Pass a list with length equal to the number of columns
        when calling get_dummies on a DataFrame. Alternatively, `prefix`
        can be a dictionary mapping column names to prefixes.
    prefix_sep : str, default '_'
        If appending prefix, separator/delimiter to use. Or pass a
        list or dictionary as with `prefix`.
    dummy_na : bool, default False
        Add a column to indicate NaNs, if False NaNs are ignored.
    columns : list-like, default None
        Colu

In [107]:
df

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2,6,5,4,XE,2279,11049,12266,1655,42.0,1736,55.0,74.0,0.0,0.0,13.0,0.0,42.0
1,2,6,12,4,XE,2379,11049,12266,1655,26.0,1736,18.0,74.0,0.0,0.0,0.0,0.0,18.0
2,2,6,16,1,XE,2379,11049,12266,1655,118.0,1736,112.0,74.0,0.0,0.0,0.0,0.0,112.0
3,2,6,18,3,XE,2379,11049,12266,1655,-9.0,1736,17.0,74.0,0.0,0.0,17.0,0.0,0.0
4,2,6,23,1,XE,2379,11049,12266,1655,44.0,1736,39.0,74.0,0.0,39.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89436,2,6,8,7,OO,6965,12892,14893,1300,20.0,1425,18.0,373.0,18.0,0.0,0.0,0.0,0.0
89437,2,6,9,1,OO,3601,14869,14113,1855,13.0,1947,30.0,150.0,30.0,0.0,0.0,0.0,0.0
89438,2,6,9,1,OO,3601,14113,14869,2015,19.0,2103,22.0,150.0,22.0,0.0,0.0,0.0,0.0
89439,2,6,9,1,OO,3613,14869,15389,1845,9.0,1941,21.0,175.0,21.0,0.0,0.0,0.0,0.0


In [57]:
sorted(list(df.columns))

['ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'ARR_DEL15',
 'ARR_DELAY',
 'ARR_DELAY_GROUP',
 'ARR_DELAY_NEW',
 'ARR_TIME',
 'ARR_TIME_BLK',
 'CANCELLATION_CODE',
 'CANCELLED',
 'CARRIER_DELAY',
 'CRS_ARR_TIME',
 'CRS_DEP_TIME',
 'CRS_ELAPSED_TIME',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'DEP_DEL15',
 'DEP_DELAY',
 'DEP_DELAY_GROUP',
 'DEP_DELAY_NEW',
 'DEP_TIME',
 'DEP_TIME_BLK',
 'DEST',
 'DEST_AIRPORT_ID',
 'DEST_AIRPORT_SEQ_ID',
 'DEST_CITY_MARKET_ID',
 'DEST_CITY_NAME',
 'DEST_STATE_ABR',
 'DEST_STATE_FIPS',
 'DEST_STATE_NM',
 'DEST_WAC',
 'DISTANCE',
 'DISTANCE_GROUP',
 'DIV1_AIRPORT',
 'DIV1_AIRPORT_ID',
 'DIV1_AIRPORT_SEQ_ID',
 'DIV1_LONGEST_GTIME',
 'DIV1_TAIL_NUM',
 'DIV1_TOTAL_GTIME',
 'DIV1_WHEELS_OFF',
 'DIV1_WHEELS_ON',
 'DIV2_AIRPORT',
 'DIV2_AIRPORT_ID',
 'DIV2_AIRPORT_SEQ_ID',
 'DIV2_LONGEST_GTIME',
 'DIV2_TAIL_NUM',
 'DIV2_TOTAL_GTIME',
 'DIV2_WHEELS_OFF',
 'DIV2_WHEELS_ON',
 'DIV3_AIRPORT',
 'DIV3_AIRPORT_ID',
 'DIV3_AIRPORT_SEQ_ID',
 'DIV3_LONGEST_GTIME',
 'DIV3_TAIL_NUM',
 'DIV

In [1]:
!pip3 install xgboost

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 1.7 MB 1.6 MB/s            
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
from sklearn import datasets
import xgboost as xgb

In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [5]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [6]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [7]:
model = xgb.train(param, D_train, steps)

In [8]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9326923076923078
Recall = 0.9326923076923078
Accuracy = 0.9333333333333333


In [9]:
from sklearn.model_selection import GridSearchCV

clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, Y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                        

In [10]:
model.dump_model('dump.raw.txt')

In [11]:
!cat dump.raw.txt.raw.txtp.raw.txt

booster[0]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=0.426589638
	2:leaf=-0.218769252
booster[1]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=-0.213294834
	2:[f2<4.94999981] yes=3,no=4,missing=3
		3:[f3<1.60000002] yes=5,no=6,missing=5
			5:leaf=0.42281881
			6:leaf=-0.0620689802
		4:[f2<5.05000019] yes=7,no=8,missing=7
			7:leaf=-0.0360000096
			8:leaf=-0.21140942
booster[2]:
0:[f2<4.75] yes=1,no=2,missing=1
	1:leaf=-0.209708765
	2:[f2<4.94999981] yes=3,no=4,missing=3
		3:[f0<6.25] yes=5,no=6,missing=5
			5:leaf=0.128571421
			6:leaf=-7.66345476e-09
		4:leaf=0.409090936
booster[3]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=0.294131458
	2:leaf=-0.195732966
booster[4]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=-0.189599976
	2:[f3<1.54999995] yes=3,no=4,missing=3
		3:[f2<4.94999981] yes=5,no=6,missing=5
			5:leaf=0.292952508
			6:leaf=-0.1117642
		4:[f2<5.05000019] yes=7,no=8,missing=7
			7:leaf=-0.0383815542
			8:leaf=-0.187781304
booster[5]:
0:[f2<4.75] yes=1,no=2,missin